<div class="titlepage">
    
## gurobipy-pandas
### Building optimization models from pandas dataframes

<br><br><br>
Simon Bowly<br>
Optimization Engineer

<br>

https://github.com/Gurobi/gurobipy-pandas#webinar
    
</div>

## gurobipy

- Python API for the [Gurobi Optimizer](https://www.gurobi.com/solutions/gurobi-optimizer/)
- Commercially licensed; developed and supported by Gurobi
- [Documentation](https://www.gurobi.com/documentation/current/refman/py_python_api_overview.html) on the Gurobi website
- Provides a translation layer from maths to Python code

$$
\begin{alignat}{3}
\max \quad & x + y + 2z \\
\mbox{s.t.} \quad & x + 2y + 3z \,\, & \le 4 \\
                  & x + y & \ge 1
\end{alignat}
$$

```python
x = m.addVar(vtype=GRB.BINARY, name="x")
...
m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)
m.addConstr(x + 2 * y + 3 * z <= 4, "c0")
...
```

## pandas

- Flexible data analysis and manipulation tool for Python
- Open source at [github.com/pandas-dev/pandas](https://github.com/pandas-dev/pandas)
- NumFOCUS sponsored project
- [Documentation](https://pandas.pydata.org/) on pydata.org
- Provides DataFrames for Python; plus I/O, analysis, plotting & more
- Standard package for analytics projects in Python

## gurobipy-pandas

- Linking package between `gurobipy` and `pandas`
- Open source at [github.com/Gurobi/gurobipy-pandas](https://github.com/Gurobi/gurobipy-pandas)
- Developed by several enthusiasts at Gurobi, with input from Princeton Consultants
- [Documentation](https://gurobi-optimization-gurobipy-pandas.readthedocs-hosted.com/en/latest) on readthedocs
- Allows users to build Gurobi models from Pandas data in a readable way

## In this session:

- Overview of `gurobipy-pandas` design
- Basic usage mechanics
    - Creating variables
    - Building expressions using series of variables and data
    - Creating constraints
    - Retrieving solutions
- A complete modelling example

## Design Principles

Optimization models define all data, variables, and constraints over indexes:

$$
\begin{alignat}{3}
\max \quad        & \sum_{i \in I} c_i x_i \\
\mbox{s.t.} \quad & \sum_{i \in I} a_i x_i \le b \\
                  & x_{i} \in \lbrace 0, 1 \rbrace & \forall i \in I \\
\end{alignat}
$$

These mathematical indices provide a clear way to structure data in code.

## Design Principles

- Pandas DataFrames and Series already define *data* over indexes

In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame(
    index=pd.RangeIndex(4, name="i"),
    columns=["a", "b"],
    data=np.random.random((4, 2)).round(2)
)
df

a     b
i            
0  0.45  0.08
1  0.57  0.48
2  0.10  0.95
3  0.05  0.43

- We need a way to *define variables* and *build constraints* over the same indexes

# Design Principles

`gurobipy-pandas` provides:

- Methods to create pandas-indexed series of variables
- Methods to build constraints from expressions
- Accessors to extract solutions as pandas structures

`pandas` provides:

- Existing algebraic/split-apply-combine logic
- Well known syntax and methods

- With pandas, we are already set up to define all our data over various indexes, aligned with the mathematical model.
- What we are missing:
    - Easy way to define/construct variables over the same indexes using pandas structures
    - Methods to transform data and variables into constraint expressions (actually, we just leverage pandas' capabilities here)
    - Easy way to take the resulting expressions and build constraints from them

# Installation and Imports

```
pip install gurobipy-pandas
```

In [2]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import gurobipy_pandas as gppd

# Handy trick for live coding, not for production
gppd.set_interactive()

# Quiet please
gp.setParam('OutputFlag', 0)

- If you're familiar with gurobipy, you'll know `model.update()` is needed to read back changes.
- `set_interactive` enforces that `update()` is called by every `gurobipy-pandas` method.
- In interactive mode you can immediately read back names and properties for variables and constraints you've created (handy for debugging).
- `OutputFlag=0` suppresses all Gurobi logging, so we can focus on the inputs and outputs (not recommended in general, as the logs can be very informative).

## Usage

- `gurobipy` is the entry point for:
    - creating models
    - starting optimization
    - constants, status codes, etc
- `gurobipy-pandas` provides accessors and functions to:
    - create sets of variables based on indexes
    - create constraints based on aligned series
    - extract solutions as series

In [3]:
# Create a gurobipy model
model = gp.Model()

- `gurobipy-pandas` purely consists of modelling functions
- All model creation and solver control uses `gurobipy`, all data handling uses `pandas`

## Creating variables

Using the free function `gppd.add_vars`:

- Creates one variable per entry in the index
- Returns a pandas `Series` of gurobipy `Var` objects
- Variable names are based on index values
- Variable attributes can be set

In [4]:
i = pd.RangeIndex(5, name="i")
i

RangeIndex(start=0, stop=5, step=1, name='i')

In [5]:
x = gppd.add_vars(model, i, name="x", vtype="B")
x

i
0    <gurobi.Var x[0]>
1    <gurobi.Var x[1]>
2    <gurobi.Var x[2]>
3    <gurobi.Var x[3]>
4    <gurobi.Var x[4]>
Name: x, dtype: object

- Here we set the variable type to binary, but otherwise no other attributes

## Creating variables

Using the `DataFrame.gppd` accessor:
- A new dataframe is returned with an appended column of Vars
- Variable attributes can be populated from dataframe columns

In [6]:
data = pd.DataFrame(index=pd.RangeIndex(3, name="i"),
                    data=[1.4, 0.2, 0.7], columns=['u'])
data

u
i     
0  1.4
1  0.2
2  0.7

In [7]:
variables = data.gppd.add_vars(model, name="y", ub="u")
variables

u                  y
i                        
0  1.4  <gurobi.Var y[0]>
1  0.2  <gurobi.Var y[1]>
2  0.7  <gurobi.Var y[2]>

- The accessor allows pandas method chaining style to be used
- Upper bound for each variable is set to the value in the given column of the DataFrame
- Variable type is continuous (the default)

# Creating Expressions

- Pandas handles this for us
- We always leverage pandas-native functions
- Common operations:
    - summation
    - arithmetic operations
    - groupby (split-apply-combine) operations
- Let's explore some common mathematical expressions

## Single indexes

Consider an index $i \in I$, some variables $x_i$, and some data $c_i$:

In [8]:
i = pd.RangeIndex(5, name="i")
i

RangeIndex(start=0, stop=5, step=1, name='i')

In [9]:
x = gppd.add_vars(model, i, name="x")
x

i
0    <gurobi.Var x[0]>
1    <gurobi.Var x[1]>
2    <gurobi.Var x[2]>
3    <gurobi.Var x[3]>
4    <gurobi.Var x[4]>
Name: x, dtype: object

In [10]:
c = pd.Series(index=i, name="c", data=np.arange(1, 6))
c

i
0    1
1    2
2    3
3    4
4    5
Name: c, dtype: int64

## Arithmetic with scalars

$$
2 x_i + 5 \quad \forall i \in I
$$

- Produce a new series on the same index
- One linear expression per entry in the index

In [11]:
2*x + 5

i
0    5.0 + 2.0 x[0]
1    5.0 + 2.0 x[1]
2    5.0 + 2.0 x[2]
3    5.0 + 2.0 x[3]
4    5.0 + 2.0 x[4]
Name: x, dtype: object

## Summation

$$
\sum_i x_i
$$

- Produce a single linear expression
- Sums the whole series over the index

In [12]:
x.sum()

<gurobi.LinExpr: x[0] + x[1] + x[2] + x[3] + x[4]>

## Arithmetic with Series

$$
c_i x_i \quad \forall i \in I
$$

- Produce a new series on the same index
- Pointwise product for each entry in the index

In [13]:
c * x

i
0        x[0]
1    2.0 x[1]
2    3.0 x[2]
3    4.0 x[3]
4    5.0 x[4]
dtype: object

## Summing the result

$$
\sum_{i \in I} c_i x_i
$$

- Produce a single linear expression
- Take our pointwise product series, sum over the index

In [14]:
(c * x).sum()

<gurobi.LinExpr: x[0] + 2.0 x[1] + 3.0 x[2] + 4.0 x[3] + 5.0 x[4]>

## Looking Familiar?

Hopefully!

Any operation you would do with data in pandas, you can do in the same way with data & variables.

- To pandas users, hopefully the above code statements come naturally.
- There's no magic here. A pandas series can store `gurobipy` native modelling objects.

## Multi-Index

- Multi-indexes allow us to add dimensions for data and variables
- Start with an example DataFrame, representing the data $p_{ij}$

In [15]:
data = pd.DataFrame({
    "i": [0, 0, 1, 2, 2],
    "j": [1, 2, 0, 0, 1],
    "p": [0.1, 0.6, 1.2, 0.4, 0.9],
}).set_index(["i", "j"])
data

p
i j     
0 1  0.1
  2  0.6
1 0  1.2
2 0  0.4
  1  0.9

## Multi-Index

- Add corresponding variables $y_{ij}$ as a Series:

In [16]:
y = gppd.add_vars(model, data, name="y")
y

i  j
0  1    <gurobi.Var y[0,1]>
   2    <gurobi.Var y[0,2]>
1  0    <gurobi.Var y[1,0]>
2  0    <gurobi.Var y[2,0]>
   1    <gurobi.Var y[2,1]>
Name: y, dtype: object

- Note that we just pass the dataframe and get back a matching index
- This is how the mathematical model is structured: 1:1 correspondence between data and variables over a given index

## Grouped summation

$$
\sum_{i \in I} y_{ij} \quad \forall j \in J
$$

- For each $j$, sum $y_{ij}$ terms over all corresponding valid $i$ values
- Produces a Series of linear expressions, indexed by $j$

In [17]:
y.groupby("j").sum()

j
0        y[1,0] + y[2,0]
1        y[0,1] + y[2,1]
2    <gurobi.Var y[0,2]>
Name: y, dtype: object

- index specified in `groupby` matches `\forall`.
- `.sum()` reduces over the remaining index levels.
- Note: *valid* $j$ values. This multi-index is sparse. Not every $(i, j)$ pair is represented.
- Pandas handles this sparsity without any special handling by the user.

## Align data on partial indexes

$$
c_j y_{ij} \quad \forall i, j
$$

- For each $y_{ij}$ and $c_j$, join on the corresponding $j$
- Pandas defines how this alignment is done
- Index *names* are important

In [18]:
c = pd.Series(index=pd.RangeIndex(3, name='j'),
              data=[1.0, 2.0, 3.0], name="c")
c

j
0    1.0
1    2.0
2    3.0
Name: c, dtype: float64

In [19]:
c * y

i  j
0  1    2.0 y[0,1]
   2    3.0 y[0,2]
1  0        y[1,0]
2  0        y[2,0]
   1    2.0 y[2,1]
dtype: object

## Pandas arithmetic 'rules'

- Pandas aligns before applying operators: see [align](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.align.html) and [add](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.add.html)
- Note that this alignment is all performed by pandas, according to pandas internal rules (joining, matching, aligning, broadcasting)

## Finally...

$$
\sum_{j \in J} c_j y_{ij} \quad \forall i \in I
$$

- Use the series `c * y`
- Apply the same groupby-aggregate operation as before
- Result is a series indexed by $i$

In [20]:
(c * y).groupby("i").sum()

i
0    2.0 y[0,1] + 3.0 y[0,2]
1                     y[1,0]
2        y[2,0] + 2.0 y[2,1]
dtype: object

- We've arrived at a general/typical form appearing in optimization models
- We have almost everything we need to build sets of constraints

# Creating Constraints

- Indexes must align between two series
- Aim to build vectorized constraints (no manual iteration)

$$
\sum_j c_j y_{ij} \le b_i \quad \forall i \in I
$$

In [21]:
(c * y).groupby("i").sum()

i
0    2.0 y[0,1] + 3.0 y[0,2]
1                     y[1,0]
2        y[2,0] + 2.0 y[2,1]
dtype: object

In [22]:
b = pd.Series(index=pd.RangeIndex(3, name="i"), data=[1, 2, 3])
b

i
0    1
1    2
2    3
dtype: int64

- We know how to build the left- and right-hand sides of the expression
- What's left is to add them to the model

## Using free functions

- Use `gppd.add_constrs` free function
- Return a series of constraint handles

In [23]:
constraints = gppd.add_constrs(  
    model,
    (c * y).groupby("i").sum(),  # left-hand side
    GRB.LESS_EQUAL,              # inequality (sense)
    b,                           # right-hand side
    name="constr",
)
constraints

i
0    <gurobi.Constr constr[0]>
1    <gurobi.Constr constr[1]>
2    <gurobi.Constr constr[2]>
Name: constr, dtype: object

- `gppd.add_constrs` adds the constraints to the model
- One constraint per index entry was added
- A series of gurobipy objects are returned (handles to the new constraints)
- The returned series is based on the same index as the inputs

## Inspecting the result

- Check linear terms using `model.getRow`
- Coefficients in the `RHS` attribute

In [24]:
constraints.apply(model.getRow)

i
0    2.0 y[0,1] + 3.0 y[0,2]
1                     y[1,0]
2        y[2,0] + 2.0 y[2,1]
Name: constr, dtype: object

In [25]:
constraints.gppd.RHS

i
0    1.0
1    2.0
2    3.0
Name: constr, dtype: float64

- `apply` should be familar: this is in general not efficient when handling data, but ok for debugging & checking purposes
- `.gppd.RHS` is the first *Series accessor* we've seen
- Series accessors extract a series of attributes from a series of `gurobipy` objects

## Missing data

- Unaligned data is filled in arithmetic operations
- Missing data is represented with `NaN`s

In [26]:
# Data on a different index
b = pd.Series(index=pd.RangeIndex(4, name="i"), data=[1, 2, 3, 4])

# An arithmetic operation would give missing values
b - (c * y).groupby("i").sum()

i
0    1.0 + -2.0 y[0,1] + -3.0 y[0,2]
1                  2.0 + -1.0 y[1,0]
2    3.0 + -1.0 y[2,0] + -2.0 y[2,1]
3                                NaN
dtype: object

## Missing data

- Unaligned or missing data is an error when adding constraints

In [27]:
try:
    gppd.add_constrs(model, (c * y).groupby("i").sum(),
                     GRB.LESS_EQUAL, b, name="constr")
except:
    import logging
    logging.exception("FAILED")

ERROR:root:FAILED
Traceback (most recent call last):
  File "/var/folders/4r/6_qcvl013q38j8jjt4txhygm0002yk/T/ipykernel_52820/3157923601.py", line 2, in <module>
    gppd.add_constrs(model, (c * y).groupby("i").sum(),
  File "/Users/bowly/workspaces/gurobipy-pandas/src/gurobipy_pandas/api.py", line 198, in add_constrs
    return add_constrs_from_series(
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/bowly/workspaces/gurobipy-pandas/src/gurobipy_pandas/constraints.py", line 74, in add_constrs_from_series
    raise KeyError("series must be aligned")
KeyError: 'series must be aligned'


- *Important point*: if the left- and right-hand sides of a constraint don't align, then some data must be missing, or a variable was not properly defined.
- This is a hard error in gurobipy pandas. The user must ensure no data is missing and indexes are aligned.

## Creating Constraints

- Using `DataFrame.gppd` accessors
- Enables method chaining

In [28]:
data = pd.DataFrame({
    "i": [0, 0, 1, 2, 2],
    "j": [1, 2, 0, 0, 1],
    "p": [0.1, 0.6, 1.2, 0.4, 0.9],
}).set_index(["i", "j"])
data

p
i j     
0 1  0.1
  2  0.6
1 0  1.2
2 0  0.4
  1  0.9

## Creating Constraints

- Use pandas `eval`-like syntax
- One constraint added per row

In [29]:
vars_and_constrs = (
    data
    .gppd.add_vars(model, name="y")
    .gppd.add_vars(model, name="z")
    .gppd.add_constrs(model, "y + z <= 1", name="c1")
)
vars_and_constrs

p                    y                    z                       c1
i j                                                                        
0 1  0.1  <gurobi.Var y[0,1]>  <gurobi.Var z[0,1]>  <gurobi.Constr c1[0,1]>
  2  0.6  <gurobi.Var y[0,2]>  <gurobi.Var z[0,2]>  <gurobi.Constr c1[0,2]>
1 0  1.2  <gurobi.Var y[1,0]>  <gurobi.Var z[1,0]>  <gurobi.Constr c1[1,0]>
2 0  0.4  <gurobi.Var y[2,0]>  <gurobi.Var z[2,0]>  <gurobi.Constr c1[2,0]>
  1  0.9  <gurobi.Var y[2,1]>  <gurobi.Var z[2,1]>  <gurobi.Constr c1[2,1]>

In [30]:
vars_and_constrs["c1"].apply(model.getRow)

i  j
0  1    y[0,1] + z[0,1]
   2    y[0,2] + z[0,2]
1  0    y[1,0] + z[1,0]
2  0    y[2,0] + z[2,0]
   1    y[2,1] + z[2,1]
Name: c1, dtype: object

## Setting the Objective

- Objectives are set from single expressions
- No `gurobipy-pandas` method here (no vectorized operations)

In [31]:
model.setObjective(y.sum(), sense=GRB.MAXIMIZE)

In [32]:
model.update()
model.getObjective()

<gurobi.LinExpr: y[0,1] + y[0,2] + y[1,0] + y[2,0] + y[2,1]>

- We need an update to read the objective back (`gppd.set_interactive()` only applies to `gurobipy-pandas` methods)

# Extracting Solutions

- In `gurobipy`, solutions are retrieved from the `.X` attribute of a `Var`
- `gppd` *Series accessor* vectorizes this operation
- Works for any attributes (bounds, coefficients, RHS, etc)
- Returns a Series on the same index

In [33]:
model.optimize()
y.gppd.X  # Series accessor

i  j
0  1    0.5
   2    0.0
1  0    2.0
2  0    3.0
   1    0.0
Name: y, dtype: float64

# Example

- [Project-Team Allocation](https://gurobi-optimization-gurobipy-pandas.readthedocs-hosted.com/en/latest/examples/projects.html)
- TODO: screenshot of docs page which presents this in more depth

# The Model

- Given a set of projects $i \in I$ and teams $j \in J$
- Project $i$ requires $w_i$ resources to complete, and each team $j$ has capacity $c_j$
- If team $j$ completes project $i$, we get profit $p_{ij}$
- Our goal is to maximize the value of completed projects, respecting team capacities

# The Data

*Before* taking any modelling steps: prepare your data properly:
- Clearly define your model indexes, and align your dataframes to these indexes
- Keep data reading & cleaning separate from model building

In [34]:
projects = pd.read_csv("projects.csv", index_col="project")
projects.head(3)

resource
project          
0             1.1
1             1.4
2             1.2

In [35]:
teams = pd.read_csv("teams.csv", index_col="team")
teams

capacity
team          
0          2.4
1          1.8
2          1.1
3          1.9
4          1.4

## Sparsity

- Note that the model is not defined over all $(i, j)$ pairs
- (Not all teams can complete all projects)
- There are 80 < 150 combinations (sparse!)
- This is tidy data which matches the model

In [36]:
project_values = pd.read_csv("project_values.csv", index_col=["project", "team"])
project_values

profit
project team        
0       4        0.4
1       4        1.3
2       0        1.7
        1        1.7
        2        1.7
...              ...
28      2        1.0
        3        1.0
        4        1.0
29      3        1.3
        4        1.3

[80 rows x 1 columns]

## Clean data

- All indexes ($i$, $j$, $ij$) are correctly represented
- There are no missing values
- Index alignment is correct
    - Every project in `project_values` has an entry in `projects`
    - Every team in `project_values` has an entry in `teams`

## Define variables and objective

$$
\begin{alignat}{2}
\max \quad        & \sum_{i \in I} \sum_{j \in J} p_{ij} x_{ij} \\
\mbox{s.t.} \quad & x_{ij} \in \lbrace 0, 1 \rbrace & \forall (i, j) \\
\end{alignat}
$$

In [37]:
model = gp.Model()
model.ModelSense = GRB.MAXIMIZE
x = gppd.add_vars(model, project_values, vtype=GRB.BINARY, obj="profit", name="x")
x.head()

project  team
0        4       <gurobi.Var x[0,4]>
1        4       <gurobi.Var x[1,4]>
2        0       <gurobi.Var x[2,0]>
         1       <gurobi.Var x[2,1]>
         2       <gurobi.Var x[2,2]>
Name: x, dtype: object

In [38]:
model.getObjective()

<gurobi.LinExpr: 0.4 x[0,4] + 1.3 x[1,4] + 1.7 x[2,0] + 1.7 x[2,1] + 1.7 x[2,2] + 1.7 x[2,3] + 1.7 x[2,4] + 1.3 x[3,4] + 1.3 x[4,0] + 1.3 x[4,1] + 1.3 x[4,2] + 1.3 x[4,3] + 1.3 x[4,4] + 1.8 x[5,0] + 1.8 x[5,1] + 1.8 x[5,2] + 1.8 x[5,3] + 1.8 x[5,4] + 1.2 x[6,0] + 1.2 x[6,1] + 1.2 x[6,2] + 1.2 x[6,3] + 1.2 x[6,4] + 0.9 x[7,3] + 0.9 x[7,4] + x[8,3] + x[8,4] + 1.2 x[9,4] + 0.8 x[10,0] + 0.8 x[10,1] + 0.8 x[10,2] + 0.8 x[10,3] + 0.8 x[10,4] + 1.3 x[11,0] + 1.3 x[11,1] + 1.3 x[11,2] + 1.3 x[11,3] + 1.3 x[11,4] + 0.8 x[12,3] + 0.8 x[12,4] + 1.5 x[13,0] + 1.5 x[13,1] + 1.5 x[13,2] + 1.5 x[13,3] + 1.5 x[13,4] + 1.7 x[14,3] + 1.7 x[14,4] + 1.3 x[15,4] + 0.3 x[16,4] + 1.2 x[17,0] + 1.2 x[17,1] + 1.2 x[17,2] + 1.2 x[17,3] + 1.2 x[17,4] + 1.3 x[18,3] + 1.3 x[18,4] + 1.8 x[19,3] + 1.8 x[19,4] + 1.6 x[20,3] + 1.6 x[20,4] + 1.1 x[21,3] + 1.1 x[21,4] + 0.4 x[22,4] + x[23,4] + 0.3 x[24,4] + x[25,0] + x[25,1] + x[25,2] + x[25,3] + x[25,4] + 1.8 x[26,4] + 0.8 x[27,3] + 0.8 x[27,4] + x[28,0] + x[28,1] + x

- Define binary variables for every valid $i$, $j$ pairing
- Assign linear objective coefficients up-front using attributes

## Capacity constraint

$$
\sum_{i \in I} w_{i} x_{ij} \le c_{j} \quad \forall j \in J
$$

In [39]:
capacity_constraints = gppd.add_constrs(
    model,
    (
        (projects["resource"] * x)
        .groupby("team").sum()
    ),
    GRB.LESS_EQUAL,
    teams["capacity"],
    name='capacity',
)
capacity_constraints

team
0    <gurobi.Constr capacity[0]>
1    <gurobi.Constr capacity[1]>
2    <gurobi.Constr capacity[2]>
3    <gurobi.Constr capacity[3]>
4    <gurobi.Constr capacity[4]>
Name: capacity, dtype: object

- Team capacity is respected
- Group by team index $j$, sum terms $w_i x_{ij}$

## Allocate once

$$
\sum_{j \in J} x_{ij} \le 1 \quad \forall i \in I
$$

In [40]:
allocate_once = gppd.add_constrs(
    model, x.groupby('project').sum(),
    GRB.LESS_EQUAL, 1.0, name="allocate_once",
)
allocate_once.head()

project
0    <gurobi.Constr allocate_once[0]>
1    <gurobi.Constr allocate_once[1]>
2    <gurobi.Constr allocate_once[2]>
3    <gurobi.Constr allocate_once[3]>
4    <gurobi.Constr allocate_once[4]>
Name: allocate_once, dtype: object

- A project is allocated at most once
- Group by the project index $i$, sum over possible team assignments

## Solutions

- Solve the model
- Get back solution values as a series on our original index

In [41]:
model.optimize()
x.gppd.X.head()

project  team
0        4       0.0
1        4       0.0
2        0      -0.0
         1       1.0
         2       0.0
Name: x, dtype: float64

In [42]:
(
    x.gppd.X.to_frame()
    .query("x >= 0.9").reset_index()
    .groupby("team").agg({"project": list})
)

project
team              
0           [4, 5]
1              [2]
2             [11]
3          [6, 29]
4     [14, 15, 26]

- Apply a little pandas magic to the outputs.
- Extract them, transform them, write them somewhere.
- *Important*: input data and output data can be handled directly as pandas objects.
- It should now be easy to read data from a SQL database or HTTP endpoint, use it to build a model, then write solutions to a different database or endpoint, all without leaving the comfort of pandas.

## More examples are available

- This was an abriged version of the project-team allocation example
- [More examples can be found in the documentation](https://gurobi-optimization-gurobipy-pandas.readthedocs-hosted.com/en/latest/examples.html)
- Contributions of examples welcome!

## Performance

[Dedicated page in the docs](https://gurobi-optimization-gurobipy-pandas.readthedocs-hosted.com/en/latest/performance.html). FIXME Key points:

- `gurobipy-pandas` won't magically make your model building code fast. It is best suited to cases where your inputs and outputs live naturally as pandas structures.
- The API provides some structure to take *well-organized and prepared data* and help you build *clearly defined models* in the pandas style.
- For large models, use `.agg(gp.quicksum)` (ommitted in the examples for clarity). TODO explain by example that this replaces `.sum()` in all cases.

# Final thoughts

- Check the examples for clean patterns to emulate:
    - Documentation lives on [readthedocs](https://gurobi-optimization-gurobipy-pandas.readthedocs-hosted.com/en/latest/index.html)
- Bugs? Feature request? Open an issue to discuss. PRs welcome:
    - [github.com/Gurobi/gurobipy-pandas](https://github.com/Gurobi/gurobipy-pandas)
- Usage questions and discussion:
    - See the [Gurobi community forum](https://support.gurobi.com/hc/en-us/community/topics/10373864542609-GitHub-Projects%3E)

# Thanks!

Questions?

https://www.gurobi.com/free-trial